In [ ]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import pyemu

In [ ]:
import helpers

helpers.process_csv_files(
    os.path.join("..", "models", "synthetic-valley-advanced-monthly")
)

In [ ]:
obs_csv_fname = os.path.join(
    "..", "models", "synthetic-valley-advanced-monthly", "raw_obs.csv"
)
assert os.path.exists(obs_csv_fname)
obsdf = pd.read_csv(obs_csv_fname, index_col=0, parse_dates=True)
obsdf

In [ ]:
working_d = "model_and_pest_files"
assert os.path.exists(working_d)

In [ ]:
pst = pyemu.Pst(os.path.join(working_d, "pest.pst"))

In [ ]:
obs = pst.observation_data

In [ ]:
prefixes = ["wt", "aq", "lake-stage", "lake-swgw", "riv-flow", "riv-swgw"]
for prefix in prefixes:
    uobs = obs.loc[obs.usecol.str.contains(prefix), :].copy()
    uobs["datetime"] = pd.to_datetime(uobs.datetime)
    for usecol in uobs.usecol.unique():
        uuobs = uobs.loc[uobs.usecol == usecol, :].copy()
        for dt, name in zip(uuobs.datetime, uuobs.obsnme):
            oval = obsdf.loc[dt, usecol]
            obs.loc[name, "obsval"] = oval

In [ ]:
obs["weight"] = 0.0
obs["standard_deviation"] = np.nan

In [ ]:
hist_prefixes = ["wt", "aq", "lake-stage"]
for prefix in hist_prefixes:
    uobs = obs.loc[obs.usecol.str.startswith(prefix), :].copy()
    print(uobs.usecol.unique())
    uobs["datetime"] = pd.to_datetime(uobs.datetime)
    hist_uobs = uobs.loc[uobs.datetime.dt.year < 2015, :]
    if "lake" in prefix:
        obs.loc[hist_uobs.obsnme, "weight"] = 1.0
        obs.loc[hist_uobs.obsnme, "standard_deviation"] = 1.0

    else:
        obs.loc[hist_uobs.obsnme, "weight"] = 2.0
        obs.loc[hist_uobs.obsnme, "standard_deviation"] = 0.5

In [ ]:
if "riv-flow" in obs.usecol.unique():
    uobs = obs.loc[obs.usecol == "riv-flow", :]
    uobs["datetime"] = pd.to_datetime(uobs.datetime)
    hist_uobs = uobs.loc[uobs.datetime.dt.year < 2015, :].copy()
    hist_uobs["standard_deviation"] = [
        max(0.1, oval * 0.2) for oval in hist_uobs.obsval.values
    ]
    print(hist_uobs)

In [ ]:
pst.nnz_obs_names

In [ ]:
pst.control_data.noptmax = 0
pst.write(os.path.join(working_d, "pest.pst"), version=2)
pyemu.os_utils.run("pestpp-ies pest.pst", cwd=working_d)